In [1]:
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
# Geospatial data analysis
import geopandas as gpd
# Data Science
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import re



# Training

## Xgboost

In [2]:
top_features_xgb = pd.read_csv('_Models/XGB/_Train_PolyLand1503_Buildings1403_XGB1803_shap_importance.csv').head(200)['feature'].tolist()
FeData = pd.read_csv("_Models/XGB/Train_PolyLand1503_Buildings1403_XGB1803.csv")
FeData = FeData[["Latitude", "Longitude", "UHI Index"]+ top_features_xgb]


local_Sentinel = pd.read_csv("Train_Data/Final/__sentinel2_LocalStats_W5_20032025.csv")

local_Landsat = pd.read_csv("Train_Data/Final/__landsat_LocalStats_W5_20032025.csv")
localStats = local_Landsat.merge(local_Sentinel, on=["Latitude", "Longitude"])


train_FEdata = localStats.merge(FeData, on=["Latitude", "Longitude"])

X, y = train_FEdata.drop(columns=['UHI Index',"Latitude", "Longitude"]), train_FEdata['UHI Index'] - 1

# scaler  = StandardScaler()

# X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X.columns = X.columns.str.translate("".maketrans({"[":"{", "]":"}","<":"^"}))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,random_state=123)
X_train

,NDVI_Landsat_local_mean,NDVI_Landsat_local_std,NDVI_Landsat_local_min,NDVI_Landsat_local_max,NDVI_Landsat_local_sum,NDVI_Landsat_local_zscore,NDVI_Landsat_local_norm_anomaly,NDVI_Landsat_local_p10,NDVI_Landsat_local_p90,NDVI_Landsat_local_hotspot_flag,...,mean_distance_m_100_200,wind_obstruction_index_200_400,median_building_fp_area_m2_400_700,tree_canopy_0_700,avg_building_orientation_deg_100_200,BSI_Landsat_mean_500_1000,grass_shrub_0_200,AHI_median_500_1000,NDISI_Landsat_std_500_1000,min_distance_m_50_100
9241,0.168395,0.041170,0.125494,0.237211,4.209874,1.671514,1.000000,0.125494,0.237211,0,...,167.557476,-128.089343,1264.595688,4.496576e+06,145.166199,0.982162,70512.866353,66.842109,0.002598,58.526325
9024,0.469923,0.176486,0.211360,0.811734,11.748083,-0.409673,0.310243,0.277254,0.732373,0,...,163.561614,-110.158831,1536.244837,3.962573e+06,85.702969,0.981846,62483.181932,3.726009,0.002547,72.201674
7199,0.095208,0.042685,0.044224,0.187195,2.380203,-0.744631,0.134292,0.051904,0.155847,0,...,156.138061,-23.392222,335.233845,2.501208e+06,77.735331,0.983197,35478.823162,1.153987,0.002436,51.355127
9010,0.469632,0.117263,0.280078,0.688189,11.740808,-0.309242,0.375613,0.342441,0.574177,0,...,170.537573,-88.999054,1765.818989,3.925872e+06,97.251934,0.982024,49102.405973,2.547803,0.002383,54.123161
8140,0.138278,0.016367,0.123649,0.162775,3.456948,-0.893820,0.000000,0.123649,0.162775,0,...,156.464233,-77.993948,878.224781,3.656803e+05,29.087399,0.987298,58433.321000,1.523961,0.003635,57.002616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9785,0.542789,0.107129,0.299312,0.645366,13.569716,-0.164018,0.652804,0.374566,0.645366,0,...,158.330933,-281.483437,1297.107997,4.309893e+06,88.072774,0.984610,59988.422624,154.555925,0.004004,55.826982
7763,0.156550,0.032213,0.093433,0.198344,3.913753,0.856181,0.864524,0.123880,0.198344,0,...,162.830586,-18.243702,462.852983,2.012488e+06,91.886521,0.983400,22761.108235,2.621525,0.003047,46.427988
5218,0.216635,0.050244,0.136457,0.276102,5.415874,1.183564,1.000000,0.136457,0.276102,0,...,156.911988,-285.907676,1491.818165,4.032927e+06,127.490572,0.986899,21521.726499,187.562465,0.003039,55.365057
1346,0.668953,0.076732,0.573097,0.850852,16.723817,0.378921,0.449788,0.573097,0.737887,0,...,174.807209,-374.570912,3310.024142,2.682128e+06,102.960706,0.989459,81846.910567,320.815157,0.003047,NaN


In [3]:
import xgboost as xgb
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# scaler = StandardScaler()
# X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
# X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

def r2_metric(y_true, y_pred):
    r2 = r2_score(y_true, y_pred)
    return r2

params = {
    'objective': 'reg:squarederror',
    'learning_rate': 0.04,
    'max_depth': 6,
    'subsample': 0.8,
    'n_estimators': 10000,
    # 'min_child_weight':5,
    'reg_lambda':1,
    # 'reg_alpha':0.01,
    'eval_metric': r2_metric  # Use custom metric here
}

xgb_model = xgb.XGBRegressor(**params)

xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_test, y_test)],  
    # eval_names=['train', 'eval'],
    verbose=10
)

# Make predictions and evaluate
predictions = xgb_model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print('Mean Squared Error:', mse)
print('Mean Absolute Error:', mae)
print('R2 Score:', r2)

[0]	validation_0-rmse:0.01579	validation_0-r2_metric:0.05566	validation_1-rmse:0.01567	validation_1-r2_metric:0.05360
[10]	validation_0-rmse:0.01201	validation_0-r2_metric:0.45356	validation_1-rmse:0.01200	validation_1-r2_metric:0.44535
[20]	validation_0-rmse:0.00954	validation_0-r2_metric:0.65569	validation_1-rmse:0.00961	validation_1-r2_metric:0.64390
[30]	validation_0-rmse:0.00786	validation_0-r2_metric:0.76628	validation_1-rmse:0.00801	validation_1-r2_metric:0.75261
[40]	validation_0-rmse:0.00671	validation_0-r2_metric:0.82964	validation_1-rmse:0.00694	validation_1-r2_metric:0.81437
[50]	validation_0-rmse:0.00589	validation_0-r2_metric:0.86885	validation_1-rmse:0.00621	validation_1-r2_metric:0.85127
[60]	validation_0-rmse:0.00533	validation_0-r2_metric:0.89250	validation_1-rmse:0.00573	validation_1-r2_metric:0.87352
[70]	validation_0-rmse:0.00488	validation_0-r2_metric:0.90973	validation_1-rmse:0.00536	validation_1-r2_metric:0.88911
[80]	validation_0-rmse:0.00450	validation_0-r2_me

In [ ]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from catboost import CatBoostRegressor

y_binned = pd.qcut(y, q=10, labels=False, duplicates='drop')
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv = skf.split(X, y_binned)

params = {
    'objective': 'reg:squarederror',
    'learning_rate': 0.01,
    # 'max_depth': 6,
    'subsample': 0.8,
    'n_estimators': 10000,
    # 'reg_lambda':0.01,
    # 'reg_alpha':0.01,
    'eval_metric': r2_metric  # Use custom metric here
}
xgb_model = xgb.XGBRegressor(**params)

cross_scores= cross_val_score(xgb_model, X, y, cv=cv, scoring='r2')
print('cross_scores: ', cross_scores)
print('mean(cross_scores): ', np.mean(cross_scores))


cross_scores:  [0.97424183 0.97319593 0.97342602 0.97105122 0.96955498 0.96949383]
mean(cross_scores):  0.9718273012172786


In [10]:
# import shap
# explainer = shap.TreeExplainer(xgb_model)
# shap_values = explainer.shap_values(X_train)
# mean_abs_shap = np.abs(shap_values).mean(axis=0)
# shap_importance = pd.DataFrame({
#             'feature': X_train.columns,
#             'importance': mean_abs_shap
#         }).sort_values(by='importance', ascending=False)
# shap_importance.head(50)


# # Get the top  features
top_features = shap_importance.head(200)['feature'].tolist()
X_train_topFE, X_test_topFE = X_train[top_features], X_test[top_features]

# #rescale entirely when retrainin on whole set
# # full_scaler  = StandardScaler()
# # X_train_topFE = pd.DataFrame(full_scaler.fit_transform(X_train_topFE), columns=top_features)
# # X_test_topFE = pd.DataFrame(full_scaler.transform(X_test_topFE), columns=top_features)

X_train_topFE

,weighted_volume_1000_1500,min_height_1000_1500,weighted_area_700_1000,EUHI_Landsat_median_250_500,height_IQR_1000_1500,num_buildings_400_700,vegetation_vertical_complexity_0_700,vegetation_vertical_complexity_0_1000,road_0_50,SI_median_75_150,...,Albedo_Landsat_mean_0_25,LST_gradient_y_median_250_500,ABI_local_zscore,MNDWI_local_min,impervious_contiguity_0_700,REIP_local_norm_anomaly,EUHI_Landsat_mean_250_500,water_cooling_potential_0_500,VGI_local_min,std_distance_m_400_700
9241,3.270610e+05,18.040000,1954.618748,2005.076700,97.911513,139,3.497350,3.410597,14910.826454,17.547893,...,0.134239,0.000000e+00,0.979425,15.533226,0.655741,0.806024,2464.372088,171.308528,0.005075,95.153227
9024,3.461855e+05,34.130000,2865.414673,2314.841870,109.223535,170,3.367039,3.340296,17263.053720,31.401863,...,0.119776,-2.328306e-10,0.099802,16.220824,0.716371,0.981299,2730.231020,159.794291,0.003604,96.526318
7199,1.015997e+05,21.210000,895.544556,3038.548012,59.220569,291,3.341844,3.293269,17841.692294,11.564041,...,0.168670,0.000000e+00,1.450216,0.042988,0.800678,0.801373,3716.868070,415.884606,0.005404,85.935649
9010,3.491260e+05,34.130000,2692.956398,2441.438004,101.737500,148,3.356902,3.354729,14976.794069,28.580481,...,0.114346,0.000000e+00,1.327093,18.313444,0.715080,0.682895,2817.345565,172.271471,0.011380,94.699087
8140,9.517752e+04,24.170000,1374.183928,4366.749673,46.630000,53,2.986135,3.114804,14457.286163,12.254335,...,0.128027,3.792291e+03,0.094835,0.207007,0.702083,0.364082,4726.124471,410.820853,0.003342,84.917522
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9785,2.907778e+05,19.029465,2085.368868,1325.279757,142.944168,134,3.180358,3.135283,7017.825270,17.643221,...,0.147382,-4.750808e+03,-0.009995,16.918566,0.537192,0.888691,2222.806206,104.228680,3.181086,95.966109
7763,9.324312e+04,22.590000,1487.632093,4014.457083,46.380000,109,3.171070,3.253403,10945.454881,7.967912,...,0.172861,-9.142181e+02,1.158011,3.736423,0.766780,0.491169,4550.667252,409.709874,0.038093,86.813498
5218,9.631030e+05,13.151638,3295.905735,1923.770812,103.100000,108,3.240451,3.280458,20553.384939,17.022407,...,0.097607,-1.205538e+03,-0.994178,17.697602,0.561406,0.259023,2092.709597,197.146352,0.005460,99.850340
1346,1.255242e+06,39.688500,4441.480917,601.669976,159.539488,63,3.310977,3.267121,19615.100012,34.357446,...,0.103308,-3.206033e+03,-0.503055,24.102401,0.388002,0.670647,527.384425,197.146352,7.700073,97.749438


In [11]:

params_xgb = {
    'objective': 'reg:squarederror',
    'learning_rate': 0.008,
    'max_depth': 9,
    'subsample': 0.8,
    'n_estimators': 15000,
    # 'reg_alpha':0.1,
    # 'reg_lambda':0.01,
    'eval_metric': r2_metric 
}

xgb_model = xgb.XGBRegressor(**params_xgb)

xgb_model.fit(
    X_train_topFE, y_train,
    eval_set=[(X_train_topFE, y_train), (X_test_topFE, y_test)],  
    # eval_names=['train', 'eval'],
    verbose=20
)

[0]	validation_0-rmse:0.01614	validation_0-r2_metric:0.01337	validation_1-rmse:0.01601	validation_1-r2_metric:0.01192
[20]	validation_0-rmse:0.01408	validation_0-r2_metric:0.24925	validation_1-rmse:0.01402	validation_1-r2_metric:0.24315
[40]	validation_0-rmse:0.01233	validation_0-r2_metric:0.42460	validation_1-rmse:0.01232	validation_1-r2_metric:0.41524
[60]	validation_0-rmse:0.01081	validation_0-r2_metric:0.55708	validation_1-rmse:0.01087	validation_1-r2_metric:0.54464
[80]	validation_0-rmse:0.00952	validation_0-r2_metric:0.65703	validation_1-rmse:0.00964	validation_1-r2_metric:0.64200
[100]	validation_0-rmse:0.00840	validation_0-r2_metric:0.73292	validation_1-rmse:0.00858	validation_1-r2_metric:0.71612
[120]	validation_0-rmse:0.00743	validation_0-r2_metric:0.79072	validation_1-rmse:0.00768	validation_1-r2_metric:0.77256
[140]	validation_0-rmse:0.00662	validation_0-r2_metric:0.83429	validation_1-rmse:0.00693	validation_1-r2_metric:0.81514
[160]	validation_0-rmse:0.00591	validation_0-r

KeyboardInterrupt: 

In [57]:
import shap
explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X_test_topFE)
mean_abs_shap = np.abs(shap_values).mean(axis=0)
shap_importance = pd.DataFrame({
            'feature': X_test_topFE.columns,
            'importance': mean_abs_shap
        }).sort_values(by='importance', ascending=False)
shap_importance.head(40)

,feature,importance
0,weighted_volume_1000_1500,0.006826
2,min_height_1000_1500,0.001334
5,height_IQR_1000_1500,0.000751
8,MNDWI_median_500_1000,0.000722
3,vegetation_vertical_complexity_0_1000,0.000686
6,EUHI_Landsat_median_250_500,0.000597
10,EUHI_Landsat_mean_250_500,0.000540
9,road_0_50,0.000467
7,vegetation_vertical_complexity_0_700,0.000454
12,height_skewness_400_700,0.000391


In [ ]:
import json
# Save the model
# catBoostModel.save_model("_Models/CatBoost/Z_Train_Satellite_Weather_PolyLandCoverIDW500Circles1503_Buildings14_Ecostress.cbm")

# Save feature names separately
with open("_Models/CatBoost/Z_Train_Satellite_Weather_PolyLandCoverIDW500Circles1503_Buildings14_Ecostress.json", "w") as f:
    json.dump(top_features, f)  # Save feature names as a JSON file

## Catboost

In [ ]:
# Weather_PD = pd.read_csv('Train_Data/Final/Training_data_PD_Weather_2025-02-18.csv')
# Weather_PD = Weather_PD.drop(columns=["datetime"])

Landsat_Train = pd.read_csv('Train_Data/Final/Landsat_Train_V26022024.csv')
Sentinel2_Train = pd.read_csv('Train_Data/Final/Sentinel2_Train_v26022025.csv')
satellite_Data = Landsat_Train.merge(Sentinel2_Train, on=["Latitude", "Longitude", "UHI Index"])

# train_FEdata = Weather_PD.merge(satellite_Data, on=["Latitude", "Longitude", "UHI Index"])

Geo_FP_train = pd.read_csv('Train_Data/Final/Train_Buildings_v14032025_NegativeMask.csv').select_dtypes("number")
# Geo_FP_train = Geo_FP_train.drop(columns=["geometry","datetime", "Region_x","Region_y", "Unnamed: 0"])
# Geo_FP_train = Geo_FP_train.drop(columns=["geometry","datetime"])
train_FEdata = satellite_Data.merge(Geo_FP_train, on=["Latitude", "Longitude", "UHI Index"])

# HVSR_df = pd.read_csv('Train_Data/Final/Train_HVSR_Elevation.csv')
# train_FEdata = train_FEdata.merge(HVSR_df, on=["Latitude", "Longitude", "UHI Index"])

Ecostress = pd.read_csv("Train_Data/Final/Train_Data_Ecostress_rolling3_20210623.csv")
Ecostress = Ecostress.drop(columns=["Unnamed: 0", "Population_Density"])
train_FEdata = train_FEdata.merge(Ecostress, on=["Latitude", "Longitude", "UHI Index"])

LandCover_Train = pd.read_csv("Train_Data/Final/Polygonized_LandCover_Train_IDW_500Circles_15032025.csv")
LandCover_Train = LandCover_Train.drop(columns=["datetime", "geometry"])
train_FEdata = train_FEdata.merge(LandCover_Train, on=["Latitude", "Longitude", "UHI Index"])

# CNN_Preds= pd.read_csv("Train_Data/Final/_Train_CNNPreds2.csv")
# train_FEdata = train_FEdata.merge(CNN_Preds, on=["Latitude", "Longitude"])


train_FEdata = train_FEdata.drop(columns=["Longitude", "Latitude", "Unnamed: 0"])
train_FEdata = train_FEdata.astype('float64')
# train_FEdata.to_csv("Train_Data/Final/Z_Train_Satellite_Weather_PolyLandCover400_Buildings14_Ecostress_HVSR.csv", index=False)

X, y = train_FEdata.drop(columns=['UHI Index']), train_FEdata['UHI Index'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=123)
X_train

,NDVI_Landsat_mean_0_25,GNDVI_Landsat_mean_0_25,EVI_Landsat_mean_0_25,NDWI_Landsat_mean_0_25,MNDWI_Landsat_mean_0_25,NDBI_Landsat_mean_0_25,BUI_Landsat_mean_0_25,NDBSI_Landsat_mean_0_25,BI_Landsat_mean_0_25,BT_Landsat_mean_0_25,...,vegetation_fragmentation_0_500,IDW_vegetation_area_0_500,vegetation_vertical_complexity_0_500,closest_impervious_distance_0_500,IDW_full_impervious_area_0_500,largest_impervious_patch_0_500,effective_albedo_0_500,impervious_contiguity_0_500,impervious_edge_density_0_500,UHI_risk_index_0_500
8430,0.075872,0.126822,0.068855,-0.126822,-0.989248,0.986319,0.910302,0.989248,0.182661,434.374121,...,0.000470,2.674862e+03,3.579036,7.105816,1.087733e+08,2.668397e+08,0.112296,0.698726,0.028531,0.466865
8893,0.189999,0.224256,0.104951,-0.224256,-0.994806,0.991773,0.801938,0.994806,0.069494,439.519925,...,0.000289,2.942731e+03,3.277351,22.665073,4.768124e+07,2.668397e+08,0.109401,0.675040,0.027375,0.460383
8785,0.402920,0.391452,0.213753,-0.391452,-0.995828,0.990030,0.587601,0.995828,0.036287,421.237903,...,0.000246,1.054433e+04,3.165674,1.877746,2.536848e+08,2.668397e+08,0.067356,0.425073,0.019622,0.402251
6183,0.165243,0.195209,0.085237,-0.195209,-0.995068,0.992531,0.827353,0.995068,0.063737,431.533942,...,0.000718,8.076645e+02,3.292678,17.489870,5.460585e+07,2.668397e+08,0.120319,0.721498,0.028189,0.467761
4418,0.348080,0.381341,0.260225,-0.381341,-0.992979,0.984710,0.635081,0.992979,0.063301,428.881281,...,0.000398,1.230555e+03,3.542687,5.130134,1.238704e+08,2.668397e+08,0.073118,0.487058,0.024163,0.421254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9785,0.548853,0.527947,0.435421,-0.527947,-0.994586,0.982455,0.433980,0.994586,0.030275,428.885274,...,0.000308,2.689012e+06,3.173421,2.873267,2.813014e+04,2.668397e+08,0.094128,0.583820,0.026820,0.439090
7763,0.159237,0.205502,0.127223,-0.205502,-0.992727,0.988851,0.829015,0.992727,0.111024,444.512911,...,0.000479,2.251495e+03,3.155937,2.219582,1.302912e+08,2.668397e+08,0.122277,0.789381,0.018741,0.493028
5218,0.219323,0.255098,0.104609,-0.255098,-0.995433,0.992209,0.772322,0.995433,0.057367,427.391601,...,0.000096,1.306608e+03,3.317263,5.373941,2.047906e+08,2.668397e+08,0.102036,0.634631,0.026264,0.451611
1346,0.669852,0.630647,0.424229,-0.630647,-0.996114,0.982235,0.311840,0.996114,0.012365,405.337677,...,0.000086,2.781779e+05,3.350872,7.290865,2.267605e+08,2.668397e+08,0.057381,0.368957,0.017857,0.389463


In [4]:
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler, QuantileTransformer

x_scaler = StandardScaler()


X_train = pd.DataFrame(x_scaler.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(x_scaler.transform(X_test), columns=X_test.columns)


# Train a CatBoostRegressor model
params = {
          'iterations': 12000,
          'learning_rate':0.01,
          'depth':6,
          'loss_function':'RMSE',
          'eval_metric':'R2',
          'subsample':0.8,
          # 'nan_mode':'Min',
          'l2_leaf_reg':20,
          "min_child_samples":5,
          # 'cat_features': cat_Features,
          # 'model_size_reg':None,
          # 'reg_lambda':10,
          # 'num_leaves':,
         }
model2 = CatBoostRegressor(**params)


model2.fit(X_train, y_train, 
           eval_set=(X_test, y_test),
                early_stopping_rounds=150,
                metric_period=5,
                plot=True)

# Evaluate the model on the test set
predictions = model2.predict(X_test)
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print('Mean Squared Error:', mse)
print('Mean Absolute Error:', mae)
print('R2 score: ', r2)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.0097212	test: 0.0096042	best: 0.0096042 (0)	total: 854ms	remaining: 2h 50m 43s
5:	learn: 0.0524649	test: 0.0526859	best: 0.0526859 (5)	total: 3.14s	remaining: 1h 44m 28s
10:	learn: 0.0926144	test: 0.0930489	best: 0.0930489 (10)	total: 5.47s	remaining: 1h 39m 19s
15:	learn: 0.1300816	test: 0.1300966	best: 0.1300966 (15)	total: 7.78s	remaining: 1h 37m 8s
20:	learn: 0.1651182	test: 0.1650344	best: 0.1650344 (20)	total: 10.1s	remaining: 1h 36m 17s
25:	learn: 0.1976944	test: 0.1970836	best: 0.1970836 (25)	total: 12.4s	remaining: 1h 35m 31s
30:	learn: 0.2281100	test: 0.2275111	best: 0.2275111 (30)	total: 14.8s	remaining: 1h 35m 25s
35:	learn: 0.2572209	test: 0.2567096	best: 0.2567096 (35)	total: 17.1s	remaining: 1h 34m 40s
40:	learn: 0.2843518	test: 0.2834984	best: 0.2834984 (40)	total: 19.4s	remaining: 1h 34m 18s
45:	learn: 0.3097784	test: 0.3089835	best: 0.3089835 (45)	total: 21.7s	remaining: 1h 33m 48s
50:	learn: 0.3338201	test: 0.3328574	best: 0.3328574 (50)	total: 23.9s	rema

In [ ]:

importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': model2.get_feature_importance()
}).sort_values(by='importance', ascending=False)

importance_df
# Get the top  features
top_features = importance_df.head(400)['feature'].tolist()
X_train_topFE, X_test_topFE = X_train[top_features], X_test[top_features]



# importance_df.head(40)
X_train_topFE

,avg_nearest_neighbor_m_700_1000,BI_Landsat_std_500_1000,depth_toBedrock,num_buildings_400_700,height_IQR_400_700,median_height_400_700,road_0_85,HVSR,Road_Density_0_85,LST_Proxy_median_250_500,...,NDCI_std_250_500,avg_shadow_length_m_700_1000,Cooling_Effectiveness_Index_0_700,bare_ground_0_120,railroad_0_120,LST_gradient_x_std_500_1000,height_IQR_700_1000,NDCI_mean_250_500,VSTR_Landsat_std_500_1000,Albedo_Landsat_median_250_500
9241,49.442133,0.053742,176.848944,140.0,29.077954,89.630473,46103.285090,1.81,0.189051,340.5,...,3.443538,149.601840,3.398802e+06,0.00000,0.000000,7935.144637,29.638658,1.564749,0.000539,0.153537
9024,59.636902,0.054165,92.056940,169.0,95.751663,127.280000,27201.938275,3.13,0.111601,338.0,...,2.638035,238.179645,2.987613e+06,0.00000,0.000000,9628.344108,92.475000,0.932970,0.000549,0.141708
7199,34.850050,0.053786,86.121067,289.0,30.160000,85.530000,45053.210267,3.31,0.184839,319.0,...,4.076127,82.405531,1.984736e+06,0.00000,0.000000,8489.177841,27.672500,1.952944,0.000307,0.149412
9010,56.545913,0.054375,92.056940,149.0,121.535000,112.000000,24376.736965,3.13,0.099908,333.5,...,2.806029,148.366443,2.979442e+06,0.00000,0.000000,9197.434393,85.860000,1.046974,0.000504,0.143183
8140,39.504710,0.047797,86.121067,52.0,23.009982,43.780000,27651.970342,3.31,0.113436,218.5,...,22.387802,78.362347,3.195853e+05,0.00000,33877.413936,12693.042802,36.186129,12.827733,0.000588,0.119010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9785,51.066825,0.060237,7.620495,135.0,39.110000,251.470000,21101.503588,25.31,0.086511,526.5,...,7.328152,282.778905,3.197623e+06,0.00000,0.000000,10416.964666,42.274671,1.532380,0.000826,0.127295
7763,42.771020,0.059093,86.121067,112.0,44.158750,65.311295,28802.052291,3.31,0.118009,263.0,...,7.048864,48.957516,1.704054e+06,0.00000,0.000000,10555.313409,35.945675,2.241165,0.000494,0.146167
5218,61.403917,0.037158,19.637130,107.0,76.912500,170.495000,41327.944823,11.44,0.169399,342.0,...,8.948065,169.238649,2.965126e+06,0.00000,0.000000,10661.379395,75.410000,2.088484,0.000737,0.113531
1346,69.017360,0.032837,19.575921,61.0,41.825000,139.020000,34677.470944,11.47,0.142300,223.5,...,46.922190,317.167006,2.023524e+06,18976.35216,0.000000,8821.201195,90.790846,38.193342,0.000750,0.094470


In [11]:
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Train a CatBoostRegressor model
params = {
          'iterations':13000,
          'learning_rate':0.05,
          'depth':9,
          'loss_function':'RMSE',
          'eval_metric':'R2',
          # 'leaf_estimation_method':"Newton",
          'subsample': 0.8523702642432592,
          # 'l2_leaf_reg':7.057448575372115,
        #   'cat_features': cat_Features,
          # 'model_size_reg':None,
          # "min_child_samples":,
          # 'reg_lambda':10,
          # 'reg_alpha':10,
          # 'num_leaves':,
         }

catBoostModel = CatBoostRegressor(**params)
catBoostModel.fit(X_train_topFE, y_train,
         eval_set=(X_test_topFE, y_test),
        #  sample_weight=weights,
         # verbose=False,
          early_stopping_rounds=150,
         metric_period= 5,
         plot=True,
         )

# Evaluate the model on the test set
# predictions = catBoostModel.predict(X_test)
# mse = mean_squared_error(y_test, predictions)
# mae = mean_absolute_error(y_test, predictions)
# r2 = r2_score(y_test, predictions)

# print('Mean Squared Error:', mse)
# print('Mean Absolute Error:', mae)
# print('R2 score: ', r2)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.0572127	test: 0.0550112	best: 0.0550112 (0)	total: 659ms	remaining: 2h 22m 50s
5:	learn: 0.2845966	test: 0.2832808	best: 0.2832808 (5)	total: 2.47s	remaining: 1h 29m 10s
10:	learn: 0.4484703	test: 0.4457098	best: 0.4457098 (10)	total: 4.32s	remaining: 1h 24m 58s
15:	learn: 0.5593849	test: 0.5528287	best: 0.5528287 (15)	total: 6.11s	remaining: 1h 22m 36s
20:	learn: 0.6374113	test: 0.6286214	best: 0.6286214 (20)	total: 7.85s	remaining: 1h 20m 51s
25:	learn: 0.6968310	test: 0.6856371	best: 0.6856371 (25)	total: 9.63s	remaining: 1h 20m 6s
30:	learn: 0.7390612	test: 0.7256843	best: 0.7256843 (30)	total: 11.3s	remaining: 1h 19m 8s
35:	learn: 0.7714008	test: 0.7566508	best: 0.7566508 (35)	total: 13.2s	remaining: 1h 19m 7s
40:	learn: 0.7996983	test: 0.7841591	best: 0.7841591 (40)	total: 15s	remaining: 1h 18m 54s
45:	learn: 0.8213878	test: 0.8052491	best: 0.8052491 (45)	total: 16.8s	remaining: 1h 18m 51s
50:	learn: 0.8370496	test: 0.8204891	best: 0.8204891 (50)	total: 18.6s	remainin

# Submission

,Longitude,Latitude,LandCover_CNN
0,-73.971665,40.788763,0.963818
1,-73.971928,40.788875,0.962844
2,-73.967080,40.789080,0.963539
3,-73.972550,40.789082,0.962020
4,-73.969697,40.787953,0.959032
...,...,...,...
1035,-73.919388,40.813803,1.035608
1036,-73.931033,40.833178,1.040250
1037,-73.934647,40.854542,1.038688
1038,-73.917223,40.815413,1.037841


In [8]:
# top_features_xgb = pd.read_csv('_Models/XGB/_Train_PolyLand1503_Buildings1403_XGB1803_shap_importance.csv').head(300)['feature'].tolist()
sub_FeData = pd.read_csv("_Models/XGB/SUB_PolyLand1503_Buildings1403_XGB1803.csv")
# FeData = FeData[["Latitude", "Longitude", "UHI Index"]+ top_features_xgb]


sub_CNN_Preds= pd.read_csv("Sub_CNN_Preds.csv")
sub_CNN_Preds["LandCover_CNN"] = 1 - sub_CNN_Preds["LandCover_CNN"]

sub_FeData = sub_CNN_Preds.merge(sub_FeData, on=["Latitude", "Longitude"])


sub_FeData = sub_FeData.fillna(-10)
sub_FeData.columns = sub_FeData.columns.str.translate("".maketrans({"[":"{", "]":"}","<":"^"}))

X_val = sub_FeData[X_test.columns]
X_val
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,random_state=123)
# X_train

,LandCover_CNN,weighted_volume_1000_1500,weighted_area_700_1000,min_height_1000_1500,vegetation_vertical_complexity_0_1000,num_buildings_400_700,height_IQR_1000_1500,EUHI_Landsat_median_250_500,vegetation_vertical_complexity_0_700,MNDWI_median_500_1000,...,MNDWI_std_500_1000,Albedo_mean_75_150,NDWI_mean_75_150,LST_gradient_y_median_75_150,LST_gradient_x_mean_25_75,UTFVI_Landsat_median_250_500,num_buildings_200_400,NDVI_mean_75_150,NDMI_Landsat_median_75_150,height_kurtosis_1000_1500
0,0.036182,1.117614e+06,2604.952806,13.151638,3.304898,108,117.280025,2051.126562,3.231441,21.881963,...,8.250844,1415.295200,18.978302,932.009321,-386.258982,0.488592,51,0.545534,-0.999969,0.955706
1,0.037156,1.092249e+06,2546.535877,13.151638,3.302638,109,118.585025,2100.013462,3.228517,22.012991,...,8.143350,1378.935570,18.551207,731.921275,-1958.936094,0.489073,55,0.496657,-0.999969,0.856927
2,0.036461,1.092608e+06,2499.582991,34.660000,3.393587,81,105.225000,1278.639667,3.343202,20.987914,...,9.831068,1038.619435,20.095435,7290.672790,-6525.143593,0.428880,42,0.549052,-0.999969,1.585755
3,0.037980,1.077531e+06,2558.058425,13.151638,3.311638,108,123.643186,2110.445655,3.230118,21.923218,...,8.152101,1213.648896,19.801191,271.179002,-125.845519,0.490232,58,0.383857,-0.999969,0.918845
4,0.040968,1.133409e+06,2898.374390,13.151638,3.315696,97,104.310000,1550.753713,3.306260,21.157313,...,9.251831,1584.860621,17.695968,2291.405751,-4575.551430,0.481558,40,0.385905,-0.999968,1.251110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,-0.035608,1.516805e+05,1232.189850,21.490000,3.202227,206,43.677500,2504.377459,3.261409,18.710571,...,6.331099,2032.370847,15.181232,1388.325382,-5224.898992,0.540382,103,0.762262,-0.999971,3.612799
1036,-0.040250,3.054175e+05,1576.775686,19.645620,3.335558,130,113.060852,1460.924789,3.398695,20.360405,...,8.301634,1438.406743,19.380591,0.000000,124.827180,0.461146,102,0.984406,-0.999973,0.547656
1037,-0.038688,2.401490e+05,1904.286207,19.029465,3.146626,149,109.930000,2791.000102,3.178360,17.473408,...,9.866520,1403.498669,19.464198,-4730.004877,1853.808290,0.474307,86,0.356064,-0.999971,1.367815
1038,-0.037841,1.497711e+05,1296.542145,21.490000,3.223554,239,37.696571,3178.720974,3.255569,18.852092,...,6.518421,1781.293269,17.530682,1715.809709,1136.646787,0.547564,104,2.696064,-0.999972,3.793486


In [ ]:

final_predictions = xgb_model.predict(X_val)
final_prediction_series = pd.Series(final_predictions)

submission_df = pd.DataFrame({'Longitude':sub_FeData['Longitude'].values, 
                                'Latitude':sub_FeData['Latitude'].values, 
                                'UHI Index':final_prediction_series.values})
submission_df.to_csv("submission_xgb_onCNN2.csv",index = False)
submission_df

,Longitude,Latitude,UHI Index
0,-73.971665,40.788763,0.961628
1,-73.971928,40.788875,0.962545
2,-73.967080,40.789080,0.963563
3,-73.972550,40.789082,0.961488
4,-73.969697,40.787953,0.958352
...,...,...,...
1035,-73.919388,40.813803,1.036223
1036,-73.931033,40.833178,1.041833
1037,-73.934647,40.854542,1.041879
1038,-73.917223,40.815413,1.037962
